### This script is a temporary script for analyzing the working memory results for the initial pilots. 
#### we will later create a full analysis pipline in the code "parse results same day encoding and testing"

In [1]:
import numpy as np
import pandas as pd 
from pathlib import Path
import matplotlib.pyplot as plt
import random
import os
import shutil
from itertools import chain
import seaborn as sns 

In [2]:
project_name_outputs = 'multicategory_working_memory'
PATH_TO_PROJECT = Path.cwd().parent.parent.parent / 'projects' / project_name_outputs
path_to_data = PATH_TO_PROJECT / 'experiment 1' / 'collected data' / 'pilot_same_day' / 'batch 2' / 'data'

In [3]:
results = []

for file in os.listdir(path_to_data):
        if file.endswith(".csv"):

            df = pd.read_csv(path_to_data/file)
            df_accuracy = df.iloc[3:63].dropna(axis=1,how='all')
            df_rating = df.iloc[63:123].dropna(axis=1,how='all')

            cols = ['workID', 'Age', 'Gender','target', 'image1', 'image2', 'correct', 'background_0', 'background_1', 'background_2', 'background_3', 'background_4', 'background_5', 'target_index','test_key_resp.keys','test_key_resp.rt','distractor_level','category','selected_dist' ]
            df_accuracy = df_accuracy[cols].reset_index(drop=True)
            df_rating = df_rating[['slider_part2.response','slider_part2.rt']].reset_index(drop=True)
            df_rating_and_accuarcy = pd.concat([df_accuracy,df_rating],axis=1)
            results.append(df_rating_and_accuarcy)



data_long_format = pd.concat(results,axis = 0)

In [ ]:
data_long_format['correct_bool'] = data_long_format['correct'] == data_long_format['test_key_resp.keys']

In [ ]:
data_long_format.groupby('workID')['correct_bool'].sum()

In [ ]:
grouped = data_long_format.groupby(['workID','distractor_level']).aggregate({'correct_bool':'sum'})
grouped_rating = data_long_format.groupby(['workID','distractor_level']).aggregate({'slider_part2.response':'mean'})

In [ ]:
grouped['level'] = [a[-1] for a in grouped.index]
grouped_rating['level'] = [a[-1] for a in grouped_rating.index]

In [ ]:
def add_connecting_lines_to_stripplot(stripplot_handle,ax):
  #get the current offsets:
  paths = stripplot_handle.collections
  #get x offsets:
  x_offsets = [path.get_offsets()[:,0] for path in paths]
  x_offsets_arrays=[np.array(offsets) for offsets in x_offsets]
  #get y offsets:
  y_offsets = [path.get_offsets()[:,1] for path in paths]
  y_offsets_arrays=[np.array(offsets) for offsets in y_offsets]
  # Connect each pair of points from the same row with a line
  for bar_ind in np.arange(0,len(x_offsets_arrays)-1):
    for i in range(len(x_offsets_arrays[0])):
      ax.plot([x_offsets_arrays[bar_ind][i], x_offsets_arrays[bar_ind+1][i]], [y_offsets_arrays[bar_ind][i], y_offsets_arrays[bar_ind+1][i]], color="gray", alpha=0.75)

In [ ]:
fig,axes=plt.subplots(nrows=1,ncols=2,figsize=[10,5])

sns.barplot(data=grouped/30,x='level',y = 'correct_bool',ax = axes[0])
handel1 = sns.stripplot(data=grouped/30,x='level',y = 'correct_bool',color='k',ax = axes[0])

sns.barplot(data=grouped_rating,x='level',y = 'slider_part2.response',ax = axes[1])
handel2 = sns.stripplot(data=grouped_rating,x='level',y = 'slider_part2.response',color='k',ax = axes[1])

#use the function with the handle that was returned from the stripplot
add_connecting_lines_to_stripplot(handel1,axes[0])
add_connecting_lines_to_stripplot(handel2,axes[1])